# News Web-Crawling

In [16]:
import requests
from bs4 import BeautifulSoup
import re
import time

# 사용자 에이전트 설정
user_agent_string = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"

# 웹 페이지 요청 및 파싱 함수
def make_request(url):
    response = requests.get(url, headers={'User-Agent': user_agent_string})
    if response.status_code == 200:
        content_type = response.headers['Content-Type'].lower()
        if "euc-kr" in content_type or ".html" in url:
            response.encoding = 'euc-kr'
        return BeautifulSoup(response.content, 'html.parser')
    else:
        return None

In [4]:
import pandas as pd

url_list_all = pd.read_csv('MediaTopic_R/article.url.csv')
url_list_all.head()

,ID,news,URL
0,1.100101e+06,경향신문,http://news.khan.co.kr/kh_news/khan_art_view.h...
1,1.100101e+06,경향신문,http://news.khan.co.kr/kh_news/khan_art_view.h...
2,1.100101e+06,경향신문,http://news.khan.co.kr/kh_news/khan_art_view.h...
3,1.100101e+06,경향신문,http://news.khan.co.kr/kh_news/khan_art_view.h...
4,1.100101e+06,경향신문,http://news.khan.co.kr/kh_news/khan_art_view.h...


In [3]:
url_list_all[url_list_all['URL'].str.contains('news.join')]

,ID,news,URL
334,1.100901e+06,중앙일보,https://news.joins.com/article/olink/3176644
335,1.100901e+06,중앙일보,https://news.joins.com/article/olink/3185500
336,1.100901e+06,중앙일보,https://news.joins.com/article/olink/3276221
337,1.100901e+06,중앙일보,https://news.joins.com/article/olink/3279497


In [5]:
url_list_all = url_list_all[url_list_all['URL'].str.contains('news.join') == False]

In [6]:
print(url_list_all['ID'].nunique())
print(url_list_all['news'].unique())

625
['경향신문' '동아일보' '중앙일보' '한겨레']


In [5]:
url_list_all['URL'].nunique()

637

## 경향신문


In [ ]:
url_list_kh = url_list_all[url_list_all['news']=='경향신문']
url_list_kh['content'] = ""
url_list_kh.head()

In [ ]:
## 
content_list = []
i = 0
for url in url_list_kh['URL']:
    webpage = make_request(url)
    if webpage:
        if webpage.select('div.article_cont'):
            article_text = ' '.join([p.get_text() for p in webpage.select('div.article_cont')])
        else:
            article_text = ' '.join([p.get_text() for p in webpage.select('div.art_body')])

        article_text = re.sub(r'\n|\r|\t', '', article_text).strip()
        article_text = re.sub(r'createIframe\(.*?\);\s*', '', article_text)
   
    url_list_kh['content'][i] = article_text
    time.sleep(1)
    print(url, url_list_kh['content'][i])
    i += 1

In [22]:
url_list_kh.to_csv('url_list_kh.csv', encoding='utf-8-sig')
url_list_kh.head()

,ID,news,URL,content
0,1.100101e+06,경향신문,http://news.khan.co.kr/kh_news/khan_art_view.h...,방송과 통신을 함께 관장하는 방송통신위원회(방통위) 설치 법안이 국회를 통과해 조만...
1,1.100101e+06,경향신문,http://news.khan.co.kr/kh_news/khan_art_view.h...,노무현 정부 출범 초기인 2003년 4월 초 우리는 본란을 통해 “노 대통령의 서동...
2,1.100101e+06,경향신문,http://news.khan.co.kr/kh_news/khan_art_view.h...,이명박 대통령이 언론계와 학계의 반대에도 불구하고 최시중씨를 방송통신위원장에 임명했...
3,1.100101e+06,경향신문,http://news.khan.co.kr/kh_news/khan_art_view.h...,이명박 정권의 KBS 장악 기도가 집요하게 진행되고 있다. KBS 이사회는 어제 임...
4,1.100101e+06,경향신문,http://news.khan.co.kr/kh_news/khan_art_view.h...,작가 이문열씨는 수많은 베스트 셀러를 펴낸 것으로 유명하지만 물불 가리지 않는 정치...


## 동아일보


In [23]:
url_list_da = url_list_all[url_list_all['news']=='동아일보'].reset_index(drop=True)
url_list_da['content'] = ""
url_list_da.head()

,ID,news,URL,content
0,1.100401e+06,동아일보,http://news.donga.com/3/all/20181207/93201590/1,
1,1.100401e+06,동아일보,http://news.donga.com/3/all/20181213/93284042/1,
2,1.100401e+06,동아일보,http://www.donga.com/news/article/all/20191104...,
3,1.100401e+06,동아일보,http://www.donga.com/news/article/all/20200213...,
4,1.100401e+06,동아일보,http://www.donga.com/news/article/all/20200510...,


In [24]:
content_list = []
for url in url_list_da['URL']:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    content = soup.select_one('.article_txt')
    article_text = re.sub(r'\n|\r|\t', '', content.text).strip()
    article_text = re.sub(r'createIframe\(.*?\);\s*', '', article_text)
    content_list.append(article_text)
    time.sleep(1)
    
url_list_da['content'] = content_list
url_list_da  

,ID,news,URL,content
0,1.100401e+06,동아일보,http://news.donga.com/3/all/20181207/93201590/1,KBS 1TV 시사프로그램 ‘오늘밤 김제동’은 4일 ‘김정은 위인맞이 환영단’의 김...
1,1.100401e+06,동아일보,http://news.donga.com/3/all/20181213/93284042/1,방송통신위원회는 12일 지상파 TV에 중간광고를 허용하는 ‘방송법 시행령’ 개정안을...
2,1.100401e+06,동아일보,http://www.donga.com/news/article/all/20191104...,지난달 31일 오후 독도에서 이륙한 중앙119 구조본부 소속 헬기가 인근 해상에서 ...
3,1.100401e+06,동아일보,http://www.donga.com/news/article/all/20200213...,MBC 간판 시사 프로그램인 ‘PD수첩’이 9억 원대 아파트 소유자를 무주택자인 것...
4,1.100401e+06,동아일보,http://www.donga.com/news/article/all/20200510...,박성제 MBC 사장이 최근 한국방송학회 주최로 열린 세미나에서 MBC도 KBS·EB...
...,...,...,...,...
109,1.100401e+06,동아일보,https://www.donga.com/news/article/all/2021040...,방송통신위원회가 그제 지상파 중간광고 허용을 포함한 방송법 시행령 개정안을 의결했다...
110,1.100401e+06,동아일보,https://www.donga.com/news/article/all/2021051...,박성제 MBC 사장이 14일 한국언론학회 학술대회에서 자사가 후원한 세션의 기조발표...
111,1.100401e+06,동아일보,https://www.donga.com/news/article/all/2021122...,방송통신위원회가 내년에 KBS의 수신료 회계분리를 추진하겠다고 새해 업무계획에서 밝...
112,1.100401e+06,동아일보,https://www.donga.com/news/article/all/2022011...,BBC 홈페이지영국 정부가 BBC 수신료를 2년간 동결하고 면허 기간이 끝나는 20...


In [25]:
url_list_da.to_csv('url_list_da.csv', encoding='utf-8-sig')
url_list_da.head()

,ID,news,URL,content
0,1.100401e+06,동아일보,http://news.donga.com/3/all/20181207/93201590/1,KBS 1TV 시사프로그램 ‘오늘밤 김제동’은 4일 ‘김정은 위인맞이 환영단’의 김...
1,1.100401e+06,동아일보,http://news.donga.com/3/all/20181213/93284042/1,방송통신위원회는 12일 지상파 TV에 중간광고를 허용하는 ‘방송법 시행령’ 개정안을...
2,1.100401e+06,동아일보,http://www.donga.com/news/article/all/20191104...,지난달 31일 오후 독도에서 이륙한 중앙119 구조본부 소속 헬기가 인근 해상에서 ...
3,1.100401e+06,동아일보,http://www.donga.com/news/article/all/20200213...,MBC 간판 시사 프로그램인 ‘PD수첩’이 9억 원대 아파트 소유자를 무주택자인 것...
4,1.100401e+06,동아일보,http://www.donga.com/news/article/all/20200510...,박성제 MBC 사장이 최근 한국방송학회 주최로 열린 세미나에서 MBC도 KBS·EB...


## 중앙일보


In [26]:
url_list_ja = url_list_all[url_list_all['news']=='중앙일보'].reset_index(drop=True)
url_list_ja['content'] = ""
url_list_ja.head()

,ID,news,URL,content
0,1.100901e+06,중앙일보,https://www.joongang.co.kr/article/11050258,
1,1.100901e+06,중앙일보,https://www.joongang.co.kr/article/14024664,
2,1.100901e+06,중앙일보,https://www.joongang.co.kr/article/14714831,
3,1.100901e+06,중앙일보,https://www.joongang.co.kr/article/14894059,
4,1.100901e+06,중앙일보,https://www.joongang.co.kr/article/15048877,


In [ ]:
content_list = []
i = 0
for url in url_list_ja['URL']:
    webpage = make_request(url)
    if webpage:
        paragraphs = webpage.find_all('p')
        texts = [p.get_text() for p in paragraphs]
        article_text = max(texts, key=len, default="")
        content_list.append(article_text)
        time.sleep(1)
    else:
        article_text = ""
        content_list.append(article_text)
        time.sleep(1)
url_list_ja['content'] = content_list
url_list_ja      


In [28]:
url_list_ja.to_csv('url_list_ja.csv', encoding='utf-8-sig')
url_list_ja.head()

,ID,news,URL,content
0,1.100901e+06,중앙일보,https://www.joongang.co.kr/article/11050258,김재철보다는 ‘김재철 이후’가 더 중요하다. MBC 대주주인 방송문화진흥회(방문진)...
1,1.100901e+06,중앙일보,https://www.joongang.co.kr/article/14024664,민주당은 당초 KBS·MBC 등 공영방송의 공정성을 확보한다는 명분으로 이를 추진...
2,1.100901e+06,중앙일보,https://www.joongang.co.kr/article/14714831,"KBS 전 보도국장은 세월호 희생자와 교통사고 사망자 비교로 물의를 빚었고, 이에..."
3,1.100901e+06,중앙일보,https://www.joongang.co.kr/article/14894059,정권이 바뀌거나 사회적으로 큰 갈등이 터질 때마다 KBS에서는 내홍이 벌어졌다. ...
4,1.100901e+06,중앙일보,https://www.joongang.co.kr/article/15048877,각계 인사 482명이 문창극 총리 후보자에 대한 KBS 보도를 강력히 비판하는 성명...


## 한겨레


In [29]:
url_list_han = url_list_all[url_list_all['news']=='한겨레'].reset_index(drop=True)
url_list_han['content'] = ""
url_list_han.head()

,ID,news,URL,content
0,1.101001e+06,한겨레,http://www.hani.co.kr/arti/opinion/editorial/1...,
1,1.101001e+06,한겨레,http://www.hani.co.kr/arti/opinion/editorial/1...,
2,1.101001e+06,한겨레,http://www.hani.co.kr/arti/opinion/editorial/1...,
3,1.101001e+06,한겨레,http://www.hani.co.kr/arti/opinion/editorial/1...,
4,1.101001e+06,한겨레,http://www.hani.co.kr/arti/opinion/editorial/1...,


In [ ]:
# url_list_han의 'url' 컬럼의 값들을 순회하면서 기사 본문을 추출
i = 0
for url in url_list_han['URL']:
    webpage = make_request(url)
    if webpage:
        if webpage.select_one('div.text') == None:
            article_elements = webpage.select('div.news_text01')
            article_text = ' '.join([element.get_text() for element in article_elements])
            article_text = re.sub(r'\n|\r|\t', '', article_text).strip()
            article_text = re.sub(r'function getPoll().*$', '', article_text)
        else:
            article_text = webpage.select_one('div.text').get_text()
            article_text = re.sub(r'\n|\r|\t', '', article_text).strip()
            article_text = re.sub(r'function getPoll().*$', '', article_text)
    else:
        article_text = ""
        time.sleep(1)
        
    url_list_han['content'][i] = article_text
    time.sleep(1)
    print(url, url_list_han['content'][i])
    i += 1

In [31]:
url_list_han.to_csv('url_list_han.csv', encoding='utf-8-sig')
url_list_han.head()

,ID,news,URL,content
0,1.101001e+06,한겨레,http://www.hani.co.kr/arti/opinion/editorial/1...,게티이미지더불어민주당이 포털사의 뉴스 편집권을 막기 위한 입법을 추진하겠다고 밝혔다...
1,1.101001e+06,한겨레,http://www.hani.co.kr/arti/opinion/editorial/1...,박병석 국회의장 주재로 31일 여의도 국회의장실에서 만난 더불어민주당 윤호중 원내대...
2,1.101001e+06,한겨레,http://www.hani.co.kr/arti/opinion/editorial/1...,윤호중 더불어민주당 원내대표(가운데)와 김기현 국민의힘 원내대표(왼쪽)가 29일 오...
3,1.101001e+06,한겨레,http://www.hani.co.kr/arti/opinion/editorial/1...,윤호중 더불어민주당 원내대표(왼쪽)와 김기현 국민의힘 원내대표(원내대표)가 29일 ...
4,1.101001e+06,한겨레,http://www.hani.co.kr/arti/opinion/editorial/1...,국민의힘 윤석열 대선 후보가 12일 오전 전주역에서 ‘열정열차’에 탑승하고 있다. ...


## 네이버뉴스

In [32]:
url_list_naver = url_list_all[url_list_all['URL'].str.contains('naver')].reset_index(drop=True)
url_list_naver['content'] = ""
url_list_naver.head()

,ID,news,URL,content
0,1.100101e+06,경향신문,https://n.news.naver.com/mnews/article/032/000...,
1,1.100101e+06,경향신문,https://n.news.naver.com/mnews/article/032/000...,
2,1.101001e+06,한겨레,https://n.news.naver.com/mnews/article/028/000...,
3,1.101001e+06,한겨레,https://n.news.naver.com/mnews/article/028/000...,
4,1.101001e+06,한겨레,https://n.news.naver.com/mnews/article/028/000...,


In [ ]:
url = url_list_naver['URL'][0]
i = 0
for url in url_list_naver['URL']:
    webpage = make_request(url)
    if webpage:
        article_elements = webpage.select('div.newsct_article')
        article_text = ' '.join([element.get_text() for element in article_elements])
        article_text = re.sub(r'\n|\r|\t', '', article_text).strip()
        article_text = re.sub(r'function getPoll().*$', '', article_text)
        
    url_list_naver['content'][i] = article_text
    time.sleep(1)
    print(url, url_list_naver['content'][i])
    i += 1

In [34]:
url_list_naver.to_csv('url_list_naver.csv', encoding='utf-8-sig')
url_list_naver.head()

,ID,news,URL,content
0,1.100101e+06,경향신문,https://n.news.naver.com/mnews/article/032/000...,KBS가 이달 중순께부터 제1라디오에 신설 예정인 노무현 대통령의 정례 라디오 주례...
1,1.100101e+06,경향신문,https://n.news.naver.com/mnews/article/032/000...,대학교수의 고발로 KBS 1TV의 한 PD가 저지른 무분별한 행동이 드러나면서 공영...
2,1.101001e+06,한겨레,https://n.news.naver.com/mnews/article/028/000...,한나라당의 도를 넘은 ‘색깔공세’가 마침내 언론인에까지 이르고 있다.한나라당 언론특...
3,1.101001e+06,한겨레,https://n.news.naver.com/mnews/article/028/000...,전국언론노동조합(언론노조)이 ‘신문개혁 총력투쟁 선포식’을 열고 입법투쟁에나섰다.언...
4,1.101001e+06,한겨레,https://n.news.naver.com/mnews/article/028/000...,[한겨레] <조선일보>와 <동아일보>가 연일 <한국방송>에 ‘색깔 공세’를 퍼붓고 ...


# Merge

In [35]:
url_list_all = pd.concat([url_list_han, url_list_ja, url_list_da, url_list_kh, url_list_naver])
url_list_all = url_list_all.reset_index(drop=True)
url_list_all

,ID,news,URL,content
0,1.101001e+06,한겨레,http://www.hani.co.kr/arti/opinion/editorial/1...,게티이미지더불어민주당이 포털사의 뉴스 편집권을 막기 위한 입법을 추진하겠다고 밝혔다...
1,1.101001e+06,한겨레,http://www.hani.co.kr/arti/opinion/editorial/1...,박병석 국회의장 주재로 31일 여의도 국회의장실에서 만난 더불어민주당 윤호중 원내대...
2,1.101001e+06,한겨레,http://www.hani.co.kr/arti/opinion/editorial/1...,윤호중 더불어민주당 원내대표(가운데)와 김기현 국민의힘 원내대표(왼쪽)가 29일 오...
3,1.101001e+06,한겨레,http://www.hani.co.kr/arti/opinion/editorial/1...,윤호중 더불어민주당 원내대표(왼쪽)와 김기현 국민의힘 원내대표(원내대표)가 29일 ...
4,1.101001e+06,한겨레,http://www.hani.co.kr/arti/opinion/editorial/1...,국민의힘 윤석열 대선 후보가 12일 오전 전주역에서 ‘열정열차’에 탑승하고 있다. ...
...,...,...,...,...
639,1.101001e+06,한겨레,https://n.news.naver.com/mnews/article/028/000...,한나라당의 도를 넘은 ‘색깔공세’가 마침내 언론인에까지 이르고 있다.한나라당 언론특...
640,1.101001e+06,한겨레,https://n.news.naver.com/mnews/article/028/000...,전국언론노동조합(언론노조)이 ‘신문개혁 총력투쟁 선포식’을 열고 입법투쟁에나섰다.언...
641,1.101001e+06,한겨레,https://n.news.naver.com/mnews/article/028/000...,[한겨레] <조선일보>와 <동아일보>가 연일 <한국방송>에 ‘색깔 공세’를 퍼붓고 ...
642,1.101001e+06,한겨레,https://n.news.naver.com/mnews/article/028/000...,[한겨레] 한나라당이 전기료와 통합·징수되는 텔레비전 수신료의 분리징수를 뼈대로 한...


In [37]:
url_list_all.to_csv('url_list_all_fixed.csv', encoding='utf-8-sig')